# Oscillink README Auto-Edit & Release Readiness Notebook

This notebook programmatically audits and (optionally) patches `README.md` to ensure it matches the release readiness criteria: SPD statement, symbol map, shapes/dtypes bullets, null-point criterion, chain prior sentence, normalized imports, provenance hash note, bundle scoring defaults, badge health, and PyPI import consistency.

Run cells sequentially. Re-running after patches should be idempotent (no further changes).

In [ ]:
# 1. Environment Setup & Imports
import difflib
import http.client
import importlib
import json
import re
import ssl
import subprocess
import sys
import textwrap
from pathlib import Path

REPO_ROOT = Path.cwd()
README_PATH = REPO_ROOT / 'README.md'

print('Repo root:', REPO_ROOT)
print('Python:', sys.version)
assert README_PATH.exists(), 'README.md not found'

In [ ]:
# 2. Load Current README
original_text = README_PATH.read_text(encoding='utf-8')
backup_path = README_PATH.with_suffix('.before_patch.md')
if not backup_path.exists():
    backup_path.write_text(original_text, encoding='utf-8')
print('Loaded README (chars):', len(original_text))
print('Backup saved to:', backup_path.name)

In [ ]:
# 3. Define Edit Snippets (SPD, Symbols, Shapes, Null-Point)
spd_snippet = textwrap.dedent('''\
SPD guarantee: With $A\\ge 0$, $B\\ge 0$ and $\\lambda_G>0$, the system matrix
$$M = \\lambda_G I + \\lambda_C L_{sym} + \\lambda_Q B + \\lambda_P L_{path} \\succ 0$$
so Jacobi-preconditioned CG converges (well-conditioned by normalized $L_{sym}$).
''')

symbol_map = textwrap.dedent('''\
### Symbol Map

| Code Param | Math | Meaning |
|------------|------|---------|
| lamG | $\\lambda_G$ | Anchor identity pull |
| lamC | $\\lambda_C$ | Graph coherence (normalized Laplacian) weight |
| lamQ | $\\lambda_Q$ | Query attraction weight |
| lamP | $\\lambda_P$ | Path (chain) prior weight |
| kneighbors | $k$ | Mutual kNN parameter |
''')

shapes_dtypes = 'Shapes & dtypes: Y: (N,D) float32, psi: (D,) float32, gates: (N,) float32 in [0,1].'

null_point_snippet = textwrap.dedent('''\
Null-point criterion: edges ranked by normalized residual energy $r_{ij} = \\lambda_C A_{ij} \\| u_i/d_i - u_j/d_j \\|^2$ after settle; top-K reported as null points.
''')

chain_prior_sentence = ('Path prior adds $\\lambda_P \\operatorname{tr}(U^T L_{path} U)$ where $L_{path}$ is the Laplacian of the user-supplied chain ( $L_{path} \\succeq 0$ preserves SPD ).')

provenance_float32_note = 'Provenance hash coerces core arrays (Y, ψ, gates) to float32 before hashing to ensure cross-platform stability.'

bundle_weights_note = 'Bundle scoring defaults: alpha=0.55 (coherence/align blend), mmr_lambda=0.25 (diversification strength).'

In [ ]:
# Helper patch functions

def ensure_block(text: str, marker_substring: str, insertion: str, anchor_pattern: str, after: bool=True) -> str:
    if marker_substring in text:
        return text
    m = re.search(anchor_pattern, text, flags=re.IGNORECASE)
    if not m:
        return text
    idx = m.end() if after else m.start()
    return text[:idx] + ("\n\n" + insertion.strip() + "\n\n") + text[idx:]

patched = original_text

# 4. SPD condition near Design principles
if 'M = \\lambda_G I + \\lambda_C L_{sym}' not in patched:
    patched = ensure_block(patched, 'M = \\lambda_G I + \\lambda_C L_{sym}', spd_snippet, r'## Design principles')

# 5. Symbol Map insertion
if '### Symbol Map' not in patched:
    # insert after Design principles section header or after SPD snippet
    if 'M = \\lambda_G I + \\lambda_C L_{sym}' in patched:
        anchor = 'M = \\lambda_G I + \\lambda_C L_{sym}'
        patched = ensure_block(patched, '### Symbol Map', symbol_map, re.escape(anchor))
    else:
        patched = ensure_block(patched, '### Symbol Map', symbol_map, r'## Design principles')

# 6. Shapes & dtypes bullet ensure in Minimal example and Advanced Gates
if '### Minimal example' in patched and shapes_dtypes not in patched:
    patched = re.sub(r'(### Minimal example\n)', r'\1\n' + shapes_dtypes + '\n\n', patched, count=1)

if 'Advanced Gates' in patched and shapes_dtypes not in patched:
    patched = re.sub(r'(### Advanced Gates.*?Optional\)\n)', r'\1' + shapes_dtypes + '\n\n', patched, flags=re.DOTALL, count=1)

# 7. Chain prior sentence
if 'Path prior adds $\\lambda_P' not in patched:
    patched = re.sub(r'(Chain Priors:.*?weakest link\).)', r'\1 ' + chain_prior_sentence, patched, count=1)

# 8. Null-point criterion line
if 'Null-point criterion:' not in patched:
    patched = ensure_block(patched, 'Null-point criterion:', null_point_snippet, r'Null-Point Diagnostics')

# 9. Normalize import examples
patched = re.sub(r'from oscillink\.core\.lattice import (OscillinkLattice)', r'from oscillink import \1', patched)

# 10. Remove stray artifact lines
patched = re.sub(r'^.*oscillink_settle_last_D.*\n?', '', patched, flags=re.MULTILINE)
patched = re.sub(r'^Example scrape excerpt:.*\n?', '', patched, flags=re.MULTILINE)

print('Patch steps 4-10 applied.')

In [ ]:
# 11. Expose & Verify Top-Level Exports
exports_ok = {}
try:
    osc = importlib.import_module('oscillink')
    for sym in ['OscillinkLattice', 'compute_diffusion_gates', 'verify_receipt']:
        exports_ok[sym] = hasattr(osc, sym)
except Exception as e:
    exports_ok['import_error'] = str(e)
print('Top-level export check:', exports_ok)

# 12. Badge Link Health Check (best-effort, skip SSL verify if needed)
badge_status = {}
ssl_ctx = ssl.create_default_context()
for line in original_text.splitlines():
    m = re.search(r'!\[[^]]*\]\((https://[^)]+)\)', line)
    if m:
        url = m.group(1)
        try:
            # crude parse
            host_path = url.replace('https://','').split('/',1)
            host = host_path[0]; path='/' + host_path[1] if len(host_path)>1 else '/'
            conn = http.client.HTTPSConnection(host, context=ssl_ctx, timeout=5)
            conn.request('HEAD', path)
            resp = conn.getresponse()
            badge_status[url] = resp.status
            conn.close()
        except Exception as e:
            badge_status[url] = f'error:{e}'
print('Badge statuses:', badge_status)

# 13. PyPI Metadata & Import Consistency Audit
pip_show = subprocess.run([sys.executable, '-m', 'pip', 'show', 'oscillink-lattice'], capture_output=True, text=True)
metadata = {}
if pip_show.returncode == 0:
    for l in pip_show.stdout.splitlines():
        if ':' in l:
            k,v = l.split(':',1)
            metadata[k.strip()] = v.strip()
import_success = exports_ok.get('OscillinkLattice', False)
print('PyPI metadata name:', metadata.get('Name'), 'version:', metadata.get('Version'))
print('Import success:', import_success)

# 14. Provenance Hash Stability Note Injection
if provenance_float32_note not in patched:
    patched = re.sub(r'(### Provenance Hash\n)', r'\1\n' + provenance_float32_note + '\n\n', patched, count=1)

# 15. Bundle Scoring Defaults Documentation Patch
if bundle_weights_note not in patched:
    patched = re.sub(r'(### Bundle Ranking Example\n)', r'\1\n' + bundle_weights_note + '\n\n', patched, count=1)

print('Sections 11-15 processed.')

In [ ]:
# 16. Generate Release Readiness Checklist (Programmatic Report)
checks = {}
checks['spd_present'] = 'M = \\lambda_G I + \\lambda_C L_{sym}' in patched
checks['symbol_map'] = '### Symbol Map' in patched
checks['shapes_dtypes'] = shapes_dtypes in patched
checks['null_point'] = 'Null-point criterion:' in patched
checks['chain_prior_sentence'] = 'Path prior adds $\\lambda_P' in patched
checks['imports_normalized'] = 'from oscillink.core.lattice' not in patched
checks['provenance_float32_note'] = provenance_float32_note in patched
checks['bundle_weights_note'] = bundle_weights_note in patched
checks['top_level_exports_ok'] = all(exports_ok.get(s) for s in ['OscillinkLattice','compute_diffusion_gates'])
checks['badges_ok'] = all((isinstance(v,int) and v<400) for v in badge_status.values()) if badge_status else True
checks['pypi_import_ok'] = import_success
print('Checklist JSON:\n', json.dumps(checks, indent=2))

# 17. Produce Diff
if patched != original_text:
    diff = difflib.unified_diff(original_text.splitlines(), patched.splitlines(), fromfile='README.md (orig)', tofile='README.md (patched)', lineterm='')
    diff_lines = list(diff)
    print('\n--- Unified Diff (first 120 lines) ---')
    for line in diff_lines[:120]:
        print(line)
    print(f'... ({len(diff_lines)} total diff lines)')
else:
    print('No differences (idempotent).')

# 18. Write Updated README.md
if patched != original_text:
    README_PATH.write_text(patched, encoding='utf-8')
    README_PATH.with_suffix('.after_patch.md').write_text(patched, encoding='utf-8')
    print('README.md updated.')
else:
    print('README.md unchanged.')

# 19. Optional: Draft docs/MATH_PRIMER.md
primer_path = REPO_ROOT / 'docs' / 'MATH_PRIMER.md'
if not primer_path.exists():
    primer_content = textwrap.dedent('''\
    # Oscillink Math Primer (Concise)

    ## Operators
    - Normalized Laplacian: $L_{sym} = D^{-1/2} (D - A) D^{-1/2}$ (mutual kNN graph)
    - Path Laplacian: $L_{path}$ over user chain edges (PSD)
    - Diffusion gating: solve $(L_{sym}+\\gamma I) h = \\beta s$ where $s_i = \max(0, \langle y_i, \psi \rangle)$.

    ## Settle System
    Stationary objective leads to SPD system:
    $$M = \\lambda_G I + \\lambda_C L_{sym} + \\lambda_Q B + \\lambda_P L_{path}$$
    Implicit Euler update (one step):
    $$(I+\Delta t M) U^{+} = U + \Delta t (\lambda_G Y + \lambda_Q B 1 \psi^T)$$

    ## Receipts
    ΔH decomposes improvements; null edges ranked by $r_{ij} = \\lambda_C A_{ij} \\| u_i/d_i - u_j/d_j \\|^2$.

    ## Parameter Hints
    - $k$: 4–12 typical
    - $\lambda_G$: 0.5–2.0
    - $\lambda_C$: 0.3–1.0
    - $\lambda_Q$: 2–8
    - $\lambda_P$: 0–0.5 (0 disables path prior)
    - Diffusion: $\gamma$ 0.05–0.3, $\beta$ 0.5–2.0
    ''').strip() + '\n'
    primer_path.write_text(primer_content, encoding='utf-8')
    print('Created', primer_path)
else:
    print('Primer already exists (skipped).')

# 20. Lightweight Unit Tests

def test_idempotent_reapply():
    second = patched
    # simulate re-running patch logic
    second2 = second  # patches guarded by presence checks
    assert second2 == second, 'Patching not idempotent'

def test_symbol_map_once():
    count = patched.count('### Symbol Map')
    assert count == 1, f'Symbol Map appears {count} times'

def test_import_normalization():
    assert 'from oscillink.core.lattice' not in patched, 'Legacy import pattern remains'

test_idempotent_reapply()
test_symbol_map_once()
test_import_normalization()
print('Lightweight tests passed.')